In [1]:
from hmmlearn import hmm
import numpy as np

In [2]:
train_X = np.load('../data/mfcc_feat/train.npy')
dev_X = np.load('../data/mfcc_feat/dev.npy')
test_X = np.load('../data/mfcc_feat/test.npy')

train_y = np.load('../data/labels/train_frame.npy')
dev_y = np.load('../data/labels/dev_frame.npy')

In [3]:
class Model:
    def __init__(self, CATEGORY=None, n_comp=1, n_mix=1, cov_type='full', n_iter=100000):
        self.n_comp = n_comp
        self.n_mix = n_mix
        self.cov_type = cov_type
        self.n_iter = n_iter
        self.CATEGORY = CATEGORY # 二分类任务
        
        self.models = [hmm.GMMHMM(n_components=self.n_comp, n_mix=self.n_mix, 
            covariance_type=self.cov_type, n_iter=self.n_iter) 
            for _ in range(len(self.CATEGORY))]
        
    def train(self, X, y):
        for k in range(len(self.CATEGORY)):
            self.models[k].fit(X[y==self.CATEGORY[k]])
    
    def predict(self, X):
        y = np.zeros((len(self.CATEGORY), X.shape[0]))

        for i in range(len(self.CATEGORY)):
            y[i] = np.array([self.models[i].score(X[idx:idx+1]) for idx in range(X.shape[0])])
        
        return y.argmax(axis=0)

In [4]:
model = Model(CATEGORY=[0, 1])

In [5]:
model.train(train_X, train_y)

In [6]:
test_pred = model.predict(test_X)

In [12]:
dev_pred = model.predict(dev_X)

In [8]:
from npy_append_array import NpyAppendArray

In [14]:
with NpyAppendArray('../results/dev_frame.npy') as npaa:
    npaa.append(dev_pred)

In [9]:
with NpyAppendArray('../results/test_frame.npy') as npaa:
    npaa.append(test_pred)